In [23]:
# Let's get our dependencies
import sys
!{sys.executable} -m pip install pandas
import pandas as pd
import shutil
from datetime import datetime
import random

In [27]:
# ----------------------------------------------------------------------------------------------------------------
# IGNORE THE CODE BLOCK BELOW IT MOCKS STUFF AND YOU'RE NOT SUPPOSED TO KNOW THAT :P
# ...
### THIS CODE IS are a replacement for EXTERNAL API'S. Assume this is actually an REST API or something like that...
### that you might call to get data from...

def get_source_A(day: str):
    """
    I am a "mostly" stateless event driven API. I return events with an "event_time"...
    
    I return the "sales" data for a given day
    
    """
    return pd.read_csv(f"source_data/sales_day_{day}.csv")
    
def get_source_B():
    """
    I am a mostly stateful API. I return the "state" as it is.
    
    I return the "user" data.
    """
    return pd.read_csv(f"source_data/users_v{random.randint(1,2)}.csv")


print("===\n Here we can sneek a peak at the sales data for day 01")
print(get_source_A("01"))
print("===\n And here's the user data")
print(get_source_B())

===
 Here we can sneek a peak at the sales data for day 01
   sales_date   quantity   revenue   user_id
0  2022-01-01          1         5         1
1  2022-01-01          2        10         2
2  2022-01-01          4        20         3
===
 And here's the user data
   user_id   name     status
0        1   sven    premium
1        2   john   standard
2        4    eve    premium


**Step 1**: As in every data warehouse, the step 1 is to get the data into the system, into some kind of "staging area". Since we're building a fully functional data warehouse here, our "staging area" is going to be immutable, or persitent. 

To do so we:

1. Call the source_A once a day, and put the results into a file called "sales_A.csv". We add a column for the "event processing time" to make sure we are able to replay the ingestion (which is just a simple select on that column). 
2. Call the source_B once a day, and snapshot the whole raw data into files "users_dayXYZ.csv".

In [35]:
# ----------------------------------------------------------------------------------------------------------------
# Data Ingestion Code
# API Data => Immutable Staging Area
def stage_source_A(today: str):
    data = get_source_A(today)
    dt = datetime.now()
    event_processing_time = datetime.timestamp(dt)
    data["event_proc_time"] = event_processing_time
    target_file = f'{event_processing_time}_data.csv'
    data.to_csv(target_file, index=False)

stage_source_A("01")    
    